# Train

In [2]:
import pasta_spaghettini_by_team as spa
#import pasta_spaghettini as spa
import pandas as pd
from tqdm import tqdm
import torch

data_route = '../../Dataset/League_of_Legends/features_tensor/'
match_list = pd.read_feather('./processed_ftr/match_result_train.ftr')
total_rows = match_list.shape[0]

model = 1 # 0: time-reversed // 1: time_reversed(bce) // 2: chronological // 3: chronological(bce)

if model == 0:
    highest_acc = 0.992985
    param_path = './parameters/param_spa_g3'
    pickle_path = f'{data_route}by_team_train/'
    bce = False
elif model == 1:
    highest_acc = 0.0
    param_path = './parameters/param_spa_g3_bce'
    pickle_path = f'{data_route}by_team_train/'
    bce = True
elif model == 2:
    highest_acc = 0.990500
    param_path = './parameters/param_spa_g3_ch'
    pickle_path = f'{data_route}by_team_train_ch/'
    bce = False
else:
    highest_acc = 0.0
    param_path = './parameters/param_spa_g3_ch_bce'
    pickle_path = f'{data_route}by_team_train_ch/'
    bce = True

PASTA = spa.PASTA(input_size=30, hidden_size=8, num_layers=1, learning_rate=0.001)
PASTA.load_parameter(param_path)

print("--Train start--")
for epoch in range(10):
    confusion_matrix, c_label = [0, 0, 0, 0], ["TN", "FP", "FN", "TP"]
    match_list = match_list.sample(frac=1).reset_index(drop=True)
    
    for idx in tqdm(range(total_rows), desc='[Epoch %1d] '%(epoch+1)):
        row = match_list.iloc[idx]
        match_id, win = row['match_no'], row['win']
        try:
            features = pd.read_pickle(f"{pickle_path}{match_id}.pkl")
        except:
            print(f'{pickle_path}{match_id}')
            break;

        winner, predict = PASTA.train(features, win, bce)
        correct = 2*winner + predict
        confusion_matrix[correct] += 1

    tn, fp, fn, tp = confusion_matrix[0], confusion_matrix[1], confusion_matrix[2], confusion_matrix[3]
    acc = (tp+tn)/(tn+fp+fn+tp)
    pre = tp/(tp+fp) if (tp+fp) != 0 else 0
    rec = tp/(tp+fn) if (tp+fn) != 0 else 0
    f1 = 2 * (pre * rec) / (pre + rec)

    for idx in range(0, 4):
        print(f'{c_label[idx]}: {confusion_matrix[idx]} / ', end='')
    print('\n' + 'Accuracy: %.6f / Precision: %.6f / Recall: %.6f / f1 score: %.6f'%(acc, pre, rec, f1))

    if acc > highest_acc:
        print(f"New record. Saving parameters...\n")
        PASTA.save_parameter(param_path)
        highest_acc = acc
    else:
        print('Save temporary parameters...\n')
        PASTA.save_parameter('./parameters/tmp')
    
    torch.cuda.empty_cache()
torch.cuda.empty_cache()

--Train start--


[Epoch 1] :   3%|▎         | 6145/200000 [01:38<51:33, 62.67it/s]


../../Dataset/League_of_Legends/features_tensor/by_team_train/210512_eun1_2827542387
TN: 2998 / FP: 41 / FN: 2437 / TP: 669 / 
Accuracy: 0.596745 / Precision: 0.942254 / Recall: 0.215390 / f1 score: 0.350629
New record. Saving parameters...



[Epoch 2] :   0%|          | 5/200000 [00:00<1:01:13, 54.44it/s]


../../Dataset/League_of_Legends/features_tensor/by_team_train/210516_eun1_2831108757
TN: 1 / FP: 0 / FN: 1 / TP: 3 / 
Accuracy: 0.800000 / Precision: 1.000000 / Recall: 0.750000 / f1 score: 0.857143
New record. Saving parameters...



[Epoch 3] :   0%|          | 15/200000 [00:00<1:04:32, 51.64it/s]


../../Dataset/League_of_Legends/features_tensor/by_team_train/210512_euw1_5257696737
TN: 9 / FP: 0 / FN: 3 / TP: 3 / 
Accuracy: 0.800000 / Precision: 1.000000 / Recall: 0.500000 / f1 score: 0.666667
Save temporary parameters...



[Epoch 4] :   0%|          | 2/200000 [00:00<1:16:40, 43.47it/s]

../../Dataset/League_of_Legends/features_tensor/by_team_train/210516_jp1_302170807


ZeroDivisionError: division by zero

# Test

In [ ]:
import pasta_spaghettini_by_team as spa
# import pasta_spaghettini as spa
import pandas as pd
from tqdm import tqdm
import torch

data_route = '../../Dataset/League_of_Legends/features_tensor/'
match_list = pd.read_feather('./processed_ftr/match_result_test.ftr')
total_rows = match_list.shape[0]

model = 1 # 0: time-reversed // 1: time_reversed(bce) // 2: chronological // 3: chronological(bce)

if model == 0:
    highest_acc = 0.992985
    param_path = './parameters/param_spa_g3'
    pickle_path = f'{data_route}by_team_test/'
    bce = False
elif model == 1:
    highest_acc = 0.0
    param_path = './parameters/param_spa_g3_bce'
    pickle_path = f'{data_route}by_team_test/'
    bce = True
elif model == 2:
    highest_acc = 0.988595
    param_path = './parameters/param_spa_g3_ch'
    pickle_path = f'{data_route}by_team_test_ch/'
    bce = False
else:
    highest_acc = 0.0
    param_path = './parameters/param_spa_g3_ch_bce'
    pickle_path = f'{data_route}by_team_test_ch/'
    bce = True

PASTA = spa.PASTA(input_size=30, hidden_size=8, num_layers=1, learning_rate=0.01)

print("--Test start--")
PASTA.load_parameter(param_path)
confusion_matrix, c_label = [0, 0, 0, 0], ["TN", "FP", "FN", "TP"]

for idx in tqdm(range(total_rows), desc='[Epoch %1d] '%(epoch+1)):
    row = match_list.iloc[idx]
    match_id, win = row['match_no'], row['win']
    features = pd.read_pickle(f"{pickle_path}{match_id}.pkl")

    winner, predict = PASTA.test(features, win, bce)
    correct = 2*winner + predict
    confusion_matrix[correct] += 1

tn, fp, fn, tp = confusion_matrix[0], confusion_matrix[1], confusion_matrix[2], confusion_matrix[3]
acc = (tp+tn)/(tn+fp+fn+tp) * 100
pre = tp/(tp+fp) * 100 if (tp+fp) != 0 else 0
rec = tp/(tp+fn) * 100 if (tp+fn) != 0 else 0
f1 = 2 * (pre * rec) / (pre + rec)

for idx in range(0, 4):
    print(f'{c_label[idx]}: {confusion_matrix[idx]} / ', end='')
print('\n' + 'Accuracy: %4.2f / Precision: %.4f / Recall: %.4f / f1 score: %.4f'%(acc, pre, rec, f1))

torch.cuda.empty_cache()